In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import datetime
import random
import time
import os
import gc

from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, confusion_matrix, classification_report
from sklearn.cluster import KMeans
from scipy.stats import mode, skew, kurtosis

from tensorflow.keras import backend as K
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from keras.models import Model

import matplotlib.pyplot as plt
import seaborn as sns

#----------
pd.options.display.max_rows = 50
pd.options.display.max_columns = 50

import warnings
warnings.simplefilter('ignore')

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv', index_col='id')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')
all_df = pd.concat([train, test]).reset_index(drop=True)

In [ ]:
cols = test.columns

In [ ]:
y = train['loss']
X = train.drop(['loss'],axis = 1)
y.shape,X.shape,test.shape

In [ ]:
minmax = StandardScaler()
X_sc = pd.DataFrame(minmax.fit_transform(X), columns=cols)
test_sc = pd.DataFrame(minmax.transform(test), columns=cols)

In [ ]:
display(pd.DataFrame(X_sc).head(2)),display(pd.DataFrame(test_sc).head(2))

In [ ]:
train_c0to20_and_c21to42 = X_sc.copy()

train_c0to9_and_c10to20 = X_sc[y.isin([i for i in range(0,21)])]
train_c0to4_and_c5to9 = X_sc[y.isin([i for i in range(0,10)])]
train_c10to14_and_c15to20 = X_sc[y.isin([i for i in range(10,21)])]

train_c21to31_and_c32to42 = X_sc[y.isin([i for i in range(21,43)])]
train_c21to25_and_c26to31 = X_sc[y.isin([i for i in range(21,32)])]
train_c32to36_and_c37to42 = X_sc[y.isin([i for i in range(32,43)])]

display(train_c0to20_and_c21to42.shape, 
        train_c0to9_and_c10to20.shape,
        train_c0to4_and_c5to9.shape,
        train_c10to14_and_c15to20.shape,
        train_c21to31_and_c32to42.shape,
        train_c21to25_and_c26to31.shape,
        train_c32to36_and_c37to42.shape)

In [ ]:
bins = [-1, 20.5, 43]
names = [0, 1]

train_c0to20_and_c21to42['target_binary'] = pd.cut(y, bins, labels=names)
display((train_c0to20_and_c21to42['target_binary']).head())
display(y.head())

bins = [-1, 9.5, 20.5, 31.5, 43]
names = [0, 1, 2, 3]

train_c0to20_and_c21to42['target_multiclass'] = pd.cut(y, bins, labels=names)
#display((train_c0to20_and_c21to42['target_multiclass']).head())
#display(y.head())

bins = [-1, 9.5, 20.5]
names = [0, 1]
train_c0to9_and_c10to20['target'] = pd.cut(y[y.isin([i for i in range(0, 21)])], bins, labels=names)
#display((train_c0to9_and_c10to20['target']).head())
#display(y[y.isin([i for i in range(0,21)])].head())

bins = [-1, 4.5, 9.5]
names = [0, 1]
train_c0to4_and_c5to9['target'] = pd.cut(y[y.isin([i for i in range(0, 10)])], bins, labels=names)

bins = [9.5, 14.5, 20.5]
names = [0, 1]
train_c10to14_and_c15to20['target'] = pd.cut(y[y.isin([i for i in range(10,21)])], bins, labels=names)

bins = [20.5, 31.5, 43]
names = [0, 1]
train_c21to31_and_c32to42['target'] = pd.cut(y[y.isin([i for i in range(21,43)])], bins, labels=names)
#display((train_c21to31_and_c32to42['target']).head())
#display(y[y.isin([i for i in range(21,43)])].head())

bins = [20.5, 25.5, 31.5]
names = [0, 1]
train_c21to25_and_c26to31['target'] = pd.cut(y[y.isin([i for i in range(21,32)])], bins, labels=names)

bins = [31.5, 36.5, 43]
names = [0, 1]
train_c32to36_and_c37to42['target'] = pd.cut(y[y.isin([i for i in range(32,43)])], bins, labels=names)

In [ ]:
train_c0to4 = X_sc[y.isin([i for i in range(0,5)])]
train_c5to9 = X_sc[y.isin([i for i in range(5,10)])]
train_c10to14 = X_sc[y.isin([i for i in range(10,15)])]
train_c15to20 = X_sc[y.isin([i for i in range(15,21)])]
train_c21to25 = X_sc[y.isin([i for i in range(21,26)])]
train_c26to31 = X_sc[y.isin([i for i in range(26,32)])]
train_c32to36 = X_sc[y.isin([i for i in range(32,36)])]
train_c37to42 = X_sc[y.isin([i for i in range(36,43)])]

In [ ]:
train_c0to4['target'] = y[y.isin([i for i in range(0,5)])]
train_c5to9['target'] = y[y.isin([i for i in range(5,10)])]
train_c10to14['target'] = y[y.isin([i for i in range(10,15)])]
train_c15to20['target'] = y[y.isin([i for i in range(15,21)])]
train_c21to25['target'] = y[y.isin([i for i in range(21,26)])]
train_c26to31['target'] = y[y.isin([i for i in range(26,32)])]
train_c32to36['target'] = y[y.isin([i for i in range(32,36)])]
train_c37to42['target'] = y[y.isin([i for i in range(36,43)])]

In [ ]:
c0to20c21to42 = train_c0to20_and_c21to42[cols].to_numpy()
t0to20t21to42_binary = train_c0to20_and_c21to42['target_binary'].to_numpy()
t0to20t21to42_multiclass = train_c0to20_and_c21to42['target_multiclass'].to_numpy()

c0to9c10to20 = train_c0to9_and_c10to20[cols].to_numpy()
t0to9t10to20 = train_c0to9_and_c10to20['target'].to_numpy()

c21to31c32to42 = train_c21to31_and_c32to42[cols].to_numpy()
t21to31t32to42 = train_c21to31_and_c32to42['target'].to_numpy()

c0to4c5to9 = train_c0to4_and_c5to9[cols].to_numpy()
t0to4t5to9 = train_c0to4_and_c5to9['target'].to_numpy()

c10to14c15to20 = train_c10to14_and_c15to20[cols].to_numpy()
t10to14t15to20 = train_c10to14_and_c15to20['target'].to_numpy()

c21to25c26to31 = train_c21to25_and_c26to31[cols].to_numpy()
t21to25t26to31 = train_c21to25_and_c26to31['target'].to_numpy()

c32to36c37to42 = train_c32to36_and_c37to42[cols].to_numpy()
t32to36t37to42 = train_c32to36_and_c37to42['target'].to_numpy()


c0to4 = train_c0to4[cols].to_numpy()
t0to4 = train_c0to4['target'].to_numpy()

c5to9 = train_c5to9[cols].to_numpy()
t5to9 = train_c5to9['target'].to_numpy()

c10to14 = train_c10to14[cols].to_numpy()
t10to14 = train_c10to14['target'].to_numpy()

c15to20 = train_c15to20[cols].to_numpy()
t15to20 = train_c15to20['target'].to_numpy()

c21to25 = train_c21to25[cols].to_numpy()
t21to25 = train_c21to25['target'].to_numpy()

c26to31 = train_c26to31[cols].to_numpy()
t26to31 = train_c26to31['target'].to_numpy()

c32to36 = train_c32to36[cols].to_numpy()
t32to36 = train_c32to36['target'].to_numpy()

c37to42 = train_c37to42[cols].to_numpy()
t37to42 = train_c37to42['target'].to_numpy()

In [ ]:
RMSE = tf.keras.metrics.RootMeanSquaredError(
    name='RMSE', dtype=None)


es = tf.keras.callbacks.EarlyStopping(
    monitor= 'val_loss', min_delta=1e-15, patience=10, verbose=1,
    mode='auto', baseline=None, restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=1,
    mode='auto')

In [ ]:
def multiclass_model():

    reg_inputs = layers.Input(shape = (100))
   
    x = layers.Dense(
            units = 2000, 
            activation ='relu',
            kernel_initializer ='he_uniform')(reg_inputs)
        
    reg_outputs = layers.Dense(
            units = 4,
        activation = 'softmax',
        kernel_initializer ='he_uniform',name = 'last')(x)

    #----------- Model instantiation  ---------------
    model = Model(reg_inputs,reg_outputs)

    return model

In [ ]:
def class_model():

    reg_inputs = layers.Input(shape = (100))
   
    x = layers.Dense(
            units = 2000, 
            activation ='relu',
            kernel_initializer ='he_uniform')(reg_inputs)
        
    reg_outputs = layers.Dense(
            units = 1,
        activation = 'sigmoid',
        kernel_initializer ='he_uniform',name = 'last')(x)

    #----------- Model instantiation  ---------------
    model = Model(reg_inputs,reg_outputs)

    return model

In [ ]:
def reg_model():

    reg_inputs = layers.Input(shape = (100))
   
    x = layers.Dense(
            units = 2000, 
            activation ='relu',
            kernel_initializer ='he_uniform')(reg_inputs)
        
    reg_outputs = layers.Dense(
            units = 1,
        activation = 'linear',
        kernel_initializer ='he_uniform',name = 'last')(x)

    #----------- Model instantiation  ---------------
    model = Model(reg_inputs,reg_outputs)

    return model

In [ ]:
K.clear_session()
history_mult = []
model_mult = multiclass_model()
model_mult.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='accuracy', optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs_all/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history_mult.append(
    model_mult.fit(
        x=c0to20c21to42,
        y=t0to20t21to42_multiclass,
        steps_per_epoch=500,
        batch_size=64,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
dfr = pd.DataFrame()
all_df_sc = pd.concat([X_sc, test_sc]).reset_index(drop=True)
all_df_sc_numpy = all_df_sc.to_numpy()

In [ ]:
multiclass_pred = pd.DataFrame(model_mult.predict(all_df_sc_numpy))
multiclass_pred

In [ ]:
K.clear_session()
history_all = []
model_all = class_model()
model_all.compile(loss=tf.keras.losses.BinaryCrossentropy(), metrics='accuracy', optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs_all/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history_all.append(
    model_all.fit(
        x=c0to20c21to42,
        y=t0to20t21to42_binary,
        steps_per_epoch=500,
        batch_size=64,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_all = pd.Series(model_all.predict(all_df_sc_numpy).flatten())

In [ ]:
dfr['Class_21to42'] = pred_all
dfr['Class_0to20'] = 1 - pred_all

In [ ]:
K.clear_session()
history0to20 = []
model0to20 = class_model()
model0to20.compile(loss=tf.keras.losses.BinaryCrossentropy(), metrics='accuracy', optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs0to20/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history0to20.append(
    model0to20.fit(
        x=c0to9c10to20,
        y=t0to9t10to20,
        steps_per_epoch=1000,
        batch_size=64,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_0to20 = pd.Series(model0to20.predict(all_df_sc_numpy).flatten())

In [ ]:
dfr['Class_10to20'] = pred_0to20
dfr['Class_0to9'] = 1 - pred_0to20

In [ ]:
K.clear_session()
history21to42 = []
model21to42 = class_model()
model21to42.compile(loss=tf.keras.losses.BinaryCrossentropy(), metrics='accuracy', optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs21to42/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history21to42.append(
    model21to42.fit(
        x=c21to31c32to42,
        y=t21to31t32to42,
        steps_per_epoch=500,
        batch_size=32,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_32to42 = pd.Series(model21to42.predict(all_df_sc_numpy).flatten())

In [ ]:
dfr['Class_32to42'] = pred_32to42
dfr['Class_21to31'] = 1 - pred_32to42

In [ ]:
K.clear_session()
history0to9 = []
model0to9 = class_model()
model0to9.compile(loss=tf.keras.losses.BinaryCrossentropy(), metrics='accuracy', optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs0to9/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history0to9.append(
    model0to9.fit(
        x=c0to4c5to9,
        y=t0to4t5to9,
        steps_per_epoch=500,
        batch_size=32,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_5to9 = pd.Series(model0to9.predict(all_df_sc_numpy).flatten())

In [ ]:
dfr['Class_5to9'] = pred_5to9
dfr['Class_0to4'] = 1 - pred_5to9

In [ ]:
K.clear_session()
history10to20 = []
model10to20 = class_model()
model10to20.compile(loss=tf.keras.losses.BinaryCrossentropy(), metrics='accuracy', optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs10to20/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history10to20.append(
    model10to20.fit(
        x=c10to14c15to20,
        y=t10to14t15to20,
        steps_per_epoch=1000,
        batch_size=64,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_15to20 = pd.Series(model10to20.predict(all_df_sc_numpy).flatten())

In [ ]:
dfr['Class_15to20'] = pred_15to20
dfr['Class_10to14'] = 1 - pred_15to20

In [ ]:
K.clear_session()
history21to31 = []
model21to31 = class_model()
model21to31.compile(loss=tf.keras.losses.BinaryCrossentropy(), metrics='accuracy', optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs21to31/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history21to31.append(
    model21to31.fit(
        x=c21to25c26to31,
        y=t21to25t26to31,
        steps_per_epoch=500,
        batch_size=32,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_26to31 = pd.Series(model21to31.predict(all_df_sc_numpy).flatten())

In [ ]:
dfr['Class_26to31'] = pred_26to31
dfr['Class_21to25'] = 1 - pred_26to31

In [ ]:
K.clear_session()
history32to42 = []
model32to42 = class_model()
model32to42.compile(loss=tf.keras.losses.BinaryCrossentropy(), metrics='accuracy', optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs32to42/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history32to42.append(
    model32to42.fit(
        x=c32to36c37to42,
        y=t32to36t37to42,
        steps_per_epoch=500,
        batch_size=32,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_37to42 = pd.Series(model32to42.predict(all_df_sc_numpy).flatten())

In [ ]:
dfr['Class_37to42'] = pred_37to42
dfr['Class_32to36'] = 1 - pred_37to42

In [ ]:
dfr

In [ ]:
result = pd.DataFrame()
result['Class_0to4'] = dfr['Class_0to20'] * dfr['Class_0to9'] * dfr['Class_0to4']
result['Class_5to9'] = dfr['Class_0to20'] * dfr['Class_0to9'] * dfr['Class_5to9']
result['Class_10to14'] = dfr['Class_0to20'] * dfr['Class_10to20'] * dfr['Class_10to14']
result['Class_15to20'] = dfr['Class_0to20'] * dfr['Class_10to20'] * dfr['Class_15to20']
result['Class_21to25'] = dfr['Class_21to42'] * dfr['Class_21to31'] * dfr['Class_21to25']
result['Class_26to31'] = dfr['Class_21to42'] * dfr['Class_21to31'] * dfr['Class_26to31']
result['Class_32to36'] = dfr['Class_21to42'] * dfr['Class_32to42'] * dfr['Class_32to36']
result['Class_37to42'] = dfr['Class_21to42'] * dfr['Class_32to42'] * dfr['Class_37to42'] 
display(result.head(10))
display(train['loss'].head(10))

In [ ]:
K.clear_session()
history0to4 = []
model0to4 = reg_model()
model0to4.compile(loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs0to4/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history0to4.append(
    model0to4.fit(
        x=c0to4,
        y=t0to4,
        steps_per_epoch=500,
        batch_size=32,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_reg = pd.DataFrame()

In [ ]:
pred_reg['0to4'] = pd.Series(model0to4.predict(all_df_sc_numpy).flatten())

In [ ]:
K.clear_session()
history5to9 = []
model5to9 = reg_model()
model5to9.compile(loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs5to9/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history5to9.append(
    model5to9.fit(
        x=c5to9,
        y=t5to9,
        steps_per_epoch=500,
        batch_size=32,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_reg['5to9'] = pd.Series(model5to9.predict(all_df_sc_numpy).flatten())

In [ ]:
K.clear_session()
history10to14 = []
model10to14 = reg_model()
model10to14.compile(loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs10to14/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history10to14.append(
    model10to14.fit(
        x=c10to14,
        y=t10to14,
        steps_per_epoch=500,
        batch_size=32,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_reg['10to14'] = pd.Series(model10to14.predict(all_df_sc_numpy).flatten())

In [ ]:
K.clear_session()
history15to20 = []
model15to20 = reg_model()
model15to20.compile(loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs15to20/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history15to20.append(
    model15to20.fit(
        x=c15to20,
        y=t15to20,
        steps_per_epoch=500,
        batch_size=32,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_reg['15to20'] = pd.Series(model15to20.predict(all_df_sc_numpy).flatten())

In [ ]:
K.clear_session()
history21to25 = []
model21to25 = reg_model()
model21to25.compile(loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs21to25/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history21to25.append(
    model21to25.fit(
        x=c21to25,
        y=t21to25,
        steps_per_epoch=500,
        batch_size=32,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_reg['21to25'] = pd.Series(model21to25.predict(all_df_sc_numpy).flatten())

In [ ]:
K.clear_session()
history26to31 = []
model26to31 = reg_model()
model26to31.compile(loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs26to31/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history26to31.append(
    model26to31.fit(
        x=c26to31,
        y=t26to31,
        steps_per_epoch=500,
        batch_size=32,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_reg['26to31'] = pd.Series(model26to31.predict(all_df_sc_numpy).flatten())

In [ ]:
K.clear_session()
history32to36 = []
model32to36 = reg_model()
model32to36.compile(loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs32to36/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history32to36.append(
    model32to36.fit(
        x=c32to36,
        y=t32to36,
        steps_per_epoch=500,
        batch_size=32,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_reg['32to36'] = pd.Series(model32to36.predict(all_df_sc_numpy).flatten())

In [ ]:
K.clear_session()
history37to42 = []
model37to42 = reg_model()
model37to42.compile(loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer = keras.optimizers.Adam(learning_rate=0.01))

log_dir = "logs37to42/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history37to42.append(
    model37to42.fit(
        x=c37to42,
        y=t37to42,
        steps_per_epoch=250,
        batch_size=8,
        epochs=100,
        validation_split=0.2,
        callbacks=[plateau, es]
    )
)

In [ ]:
pred_reg['37to42'] = pd.Series(model37to42.predict(all_df_sc_numpy).flatten())

In [ ]:
display(pred_reg)
display(result)
display(train['loss'])

In [ ]:
loss = pd.DataFrame([0 for i in range(0, 400000)], columns=['loss'])
for i in range(0, 400000):
    max_col = np.argmax(result.iloc[i, :])
    #print(max_col)
    loss.iloc[i] = pred_reg.iloc[i, max_col] 

display(loss)
display(train['loss'])

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(train['loss'], loss['loss'].iloc[:train.shape[0]], squared=False)

In [ ]:
sub['loss'] = loss['loss'].iloc[train.shape[0]:].values
sub = sub.set_index('id')
sub

In [ ]:
csv = 'nn_classification+reg_range0to4_5to9_10to15_16to20_21to25_26to31_32to36_37to42.csv'
sub.to_csv(csv)

In [ ]:
from IPython.display import FileLink
FileLink(csv)